In [3]:
tf.compat.v1.enable_eager_execution() 

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [4]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [5]:
targets = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices((df.values,targets.values))

In [6]:
ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [7]:
from collections import Counter

tokenizer = tfds.features.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)

print('Vocab-size:', len(token_counts))

Vocab-size: 87878


In [8]:
encoder = tfds.features.text.TokenTextEncoder(token_counts)

In [9]:
def encode(text_tensor,label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

In [10]:
def encode_map_fn(text,label):
    return tf.py_function(encode, inp=[text,label], Tout=(tf.int64, tf.int64))

In [11]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

In [12]:
train_data = ds_train.padded_batch(32, padded_shapes=([-1],[]))
valid_data = ds_valid.padded_batch(32, padded_shapes=([-1],[]))
test_data = ds_test.padded_batch(32, padded_shapes=([-1],[]))

In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

embedding_dim = 20
vocab_size = len(token_counts) + 2

bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embedding_dim, name = 'embed-layer'),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, name='lstm-layer'), name='bidir-lstm'),
    
    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

In [15]:
history = bi_lstm_model.fit(
    train_data,
    validation_data = valid_data,
    epochs = 10)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
625/625 [==============================] - 10691s 17s/step - loss: 0.5465 - acc: 0.7071 - val_loss: 0.5495 - val_acc: 0.7218
Epoch 2/10
625/625 [==============================] - 14402s 23s/step - loss: 0.3403 - acc: 0.8584 - val_loss: 0.4016 - val_acc: 0.8358
Epoch 3/10
625/625 [==============================] - 14116s 23s/step - loss: 0.1851 - acc: 0.9337 - val_loss: 0.3263 - val_acc: 0.8798
Epoch 4/10
625/625 [==============================] - 14173s 23s/step - loss: 0.1117 - acc: 0.9628 - val_loss: 0.3783 - val_acc: 0.8732
Epoch 5/10
625/625 [==============================] - 13327s 21s/step - loss: 0.0789 - acc: 0.9750 - val_loss: 0.5457 - val_acc: 0.8424
Epoch 6/10
625/625 [==============================] - 14403s 23s/step - loss: 0.0843 - acc: 0.9729 - val_loss: 0.6785 - val_acc: 0.8172
Epoch 7/10
625/625 [==============================] - 14717s 24s/step - loss: 0.0442 - acc